Ref doc link-https://huggingface.co/docs/transformers/tasks/translation#translation

### TODO Recording:

- Please upload the spa.txt file to the Colab runtime

In [ ]:
pip install transformers datasets evaluate sacrebleu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 10.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 13.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.3/106.3 kB 16.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 19.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 21.4 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 20.8 MB/s eta 0:00:00


Importing required libraries

In [ ]:
import numpy as np
import pandas as pd

Loading the translation pair data..link-https://www.kaggle.com/datasets/tejasurya/eng-spanish?select=spa.txt

In [ ]:
eng_spa_data = pd.read_csv("spa.txt", delimiter = "\t", header = None, names = ["Eng", "Spa"])

eng_spa_data.sample(10)

Eng  \
87567           Don't distract me while I am studying.   
103567  Have you ever stirred your coffee with a fork?   
422                                          Who fell?   
71129                 Wait till the light turns green.   
12326                                What happens now?   
91001          It was careless of you to lose the key.   
33762                          Tom handed Mary a book.   
104863  You will soon adapt yourself to this new life.   
44063                       Her hair didn't look dyed.   
102046   French and Arabic are spoken in this country.   

                                                     Spa  
87567            No me molestes cuando estoy estudiando.  
103567  ¿Alguna vez has revuelto tu café con un tenedor?  
422                                     ¿Quién se calló?  
71129            Espera a que el semáforo esté en verde.  
12326                                   ¿Qué pasa ahora?  
91001        Fue descuidado de tu parte perder la llave.  
33762                       Tom le pasó un libro a Mary.  
104863            Pronto te adaptarás a esta nueva vida.  
44063                         Su pelo no se veía teñido.  
102046             Se habla francés y árabe en ese país.

In [ ]:
eng_spa_data.shape

(118964, 2)

In [ ]:
eng_spa_data = eng_spa_data.sample(frac = 0.6)

eng_spa_data.shape

(71378, 2)

Adding index column

In [ ]:
eng_spa_data['id'] = range(len(eng_spa_data))

eng_spa_data.sample(5)

Eng  \
105398  Is there anything to drink in the refrigerator?   
14953                                Tom retired at 65.   
44616                        I love both cats and dogs.   
16583                               I like that person.   
54591                      Tom didn't pull the trigger.   

                                                 Spa     id  
105398       ¿En el frigorífico hay algo para beber?   1419  
14953                        Tom se jubiló a los 65.  58518  
44616   Me encantan tanto los perros como los gatos.  49980  
16583                          Esa persona me gusta.  61350  
54591                      Tom no apretó el gatillo.   7762

Data is made in a dictionary format with translation pairs

In [ ]:
translation_dicts = []

for index, row in eng_spa_data.iterrows():
    translation_dict = {
        "id": row["id"],
        "translation": {
            "en": row["Eng"],
            "spa": row["Spa"]
        }
    }

    translation_dicts.append(translation_dict)

translation_dicts[:10]

[{'id': 0,
  'translation': {'en': 'May I watch TV now?',
   'spa': '¿Ya puedo ver la televisión?'}},
 {'id': 1,
  'translation': {'en': 'Tom looked at the menu.',
   'spa': 'Tom miró el menú.'}},
 {'id': 2,
  'translation': {'en': 'At first, I thought I was sick.',
   'spa': 'Al comienzo, pensé que estaba enferma.'}},
 {'id': 3,
  'translation': {'en': 'He has a few friends in this town.',
   'spa': 'Tiene algunos amigos en esta ciudad.'}},
 {'id': 4, 'translation': {'en': "You don't say.", 'spa': 'No me digas.'}},
 {'id': 5,
  'translation': {'en': 'You have something of mine.',
   'spa': 'Tienes algo que es mío.'}},
 {'id': 6,
  'translation': {'en': 'Tom lives 10 miles from the Canadian border.',
   'spa': 'Tom vive a 10 millas de la frontera con Canadá.'}},
 {'id': 7,
  'translation': {'en': "I'm not the only one who did that.",
   'spa': 'No soy la única persona que hizo eso.'}},
 {'id': 8,
  'translation': {'en': 'You may not be as lucky next time.',
   'spa': 'Puede que no seas

Huggingface dataset is created from dictionary format

In [ ]:
from datasets import Dataset

eng_spa_ds = Dataset.from_list(translation_dicts)

eng_spa_ds

Dataset({
    features: ['id', 'translation'],
    num_rows: 71378
})

Data is split into training and test sets

In [ ]:
eng_spa_ds = eng_spa_ds.train_test_split(test_size = 0.2)

eng_spa_ds

DatasetDict({
    train: Dataset({
        features: ['id', 'translation'],
        num_rows: 57102
    })
    test: Dataset({
        features: ['id', 'translation'],
        num_rows: 14276
    })
})

The next step is to loading  a T5 tokenizer to process the English-Spanish language pairs

In [ ]:
from transformers import AutoTokenizer

checkpoint = "t5-small"

tokenizer = AutoTokenizer.from_pretrained(checkpoint)

tokenizer

tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

T5TokenizerFast(name_or_path='t5-small', vocab_size=32100, model_max_length=512, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'eos_token': '</s>', 'unk_token': '<unk>', 'pad_token': '<pad>', 'additional_special_tokens': ['<extra_id_0>', '<extra_id_1>', '<extra_id_2>', '<extra_id_3>', '<extra_id_4>', '<extra_id_5>', '<extra_id_6>', '<extra_id_7>', '<extra_id_8>', '<extra_id_9>', '<extra_id_10>', '<extra_id_11>', '<extra_id_12>', '<extra_id_13>', '<extra_id_14>', '<extra_id_15>', '<extra_id_16>', '<extra_id_17>', '<extra_id_18>', '<extra_id_19>', '<extra_id_20>', '<extra_id_21>', '<extra_id_22>', '<extra_id_23>', '<extra_id_24>', '<extra_id_25>', '<extra_id_26>', '<extra_id_27>', '<extra_id_28>', '<extra_id_29>', '<extra_id_30>', '<extra_id_31>', '<extra_id_32>', '<extra_id_33>', '<extra_id_34>', '<extra_id_35>', '<extra_id_36>', '<extra_id_37>', '<extra_id_38>', '<extra_id_39>', '<extra_id_40>', '<extra_id_41>', '<extra_id_42>', '<extra_id_43>', '<extra_i

The preprocessing function we want to  create needs to:

Prefix the input with a prompt so T5 knows this is a translation task. Some models capable of multiple NLP tasks require prompting for specific tasks.
Tokenize the input (English) and target (French) separately because you can’t tokenize French text with a tokenizer pretrained on an English vocabulary.
Truncate sequences to be no longer than the maximum length set by the max_length parameter.

In [ ]:
source_lang = "en"
target_lang = "spa"

prefix = "translate English to Spanish: "

def preprocess_function(examples):
    inputs = [prefix + example[source_lang] for example in examples["translation"]]
    targets = [example[target_lang] for example in examples["translation"]]

    model_inputs = tokenizer(inputs, text_target = targets, max_length = 128, truncation = True)

    return model_inputs

To apply the preprocessing function over the entire dataset, we use 🤗 Datasets map method. We can  speed up the map function by setting batched=True to process multiple elements of the dataset at once

In [ ]:
tokenized_eng_spa_ds = eng_spa_ds.map(preprocess_function, batched = True)

tokenized_eng_spa_ds

Map:   0%|          | 0/57102 [00:00<?, ? examples/s]

Map:   0%|          | 0/14276 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['id', 'translation', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 57102
    })
    test: Dataset({
        features: ['id', 'translation', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 14276
    })
})

Now creating a batch of examples using DataCollatorForSeq2Seq. It’s more efficient to dynamically pad the sentences to the longest length in a batch during collation, instead of padding the whole dataset to the maximum length.

In [ ]:
from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(
    tokenizer = tokenizer, model = checkpoint, label_pad_token_id=tokenizer.pad_token_id, return_tensors = "tf")

Including a metric during training is often helpful for evaluating your model’s performance. We can quickly load a evaluation method with the 🤗 Evaluate library. For this task, load the SacreBLEU metric.

In [ ]:
import evaluate

metric = evaluate.load("sacrebleu")

We will check our evaluation metric of sacred bleu with some simple examples.

- predictions (list of str): list of translations to score. Each translation should be tokenized into a list of tokens.
- references (list of list of str): A list of lists of references. The contents of the first sub-list are the references for the first prediction, the contents of the second sub-list are for the second prediction, etc. Note that there must be the same number of references for each prediction (i.e. all sub-lists must be of the same length).

### TODO Recording:

- Record the next two cells by pasting in one prediction at a time

#### Score-0
predictions = ["This is not cool", "Movie was terrible"]

#### Score-79
predictions = ["Where are you going", "How are you doing?"]

#### Score-100
predictions = ["Hi John, Where are you?", "How are you doing?"]

In [ ]:
predictions = ["This is not cool", "Movie was terrible"]

references = [["Hi John, Where are you?", "hello there!"], ["How are you doing?", "How are you doing?"]]

In [ ]:
results = metric.compute(predictions = predictions, references = references)

results

{'score': 0.0,
 'counts': [0, 0, 0, 0],
 'totals': [7, 5, 3, 1],
 'precisions': [0.0, 0.0, 0.0, 0.0],
 'bp': 0.8668778997501817,
 'sys_len': 7,
 'ref_len': 8}

Then creating a function that passes your predictions and labels to compute to calculate the SacreBLEU score

In [ ]:
def postprocess_text(preds, labels):
    preds = [pred.strip() for pred in preds]
    labels = [[label.strip()] for label in labels]

    return preds, labels

In this context, labels typically represents an array of label indices for a batch of data. The value -100 is often used in Hugging Face models to indicate tokens that should be ignored in loss calculations. This could be because they are padding tokens, or in the case of MLM, tokens that are not masked and therefore not predicted by the model.

Replacing -100 with tokenizer.pad_token_id: This line effectively replaces all instances of -100 in the labels array with the padding token ID of the tokenizer. The padding token ID is a special token index used to fill out sequences to a uniform length.

Significance in Model Training: This operation is significant because it prepares the label data for use in training the model. By replacing -100 with the padding token ID, it ensures that the loss function ignores these tokens (as most implementations of loss functions in deep learning frameworks automatically ignore the padding token index). This is crucial for correctly computing the loss and for the model to learn effectively.

In [ ]:
def compute_metrics(eval_preds):
    preds, labels = eval_preds

    if isinstance(preds, tuple):
        preds = preds[0]

    preds = np.where(preds != -100, preds, tokenizer.pad_token_id)
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)

    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)

    result = metric.compute(predictions=decoded_preds, references=decoded_labels)
    result = {"bleu": result["score"]}

    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in preds]

    result["gen_len"] = np.mean(prediction_lens)

    result = {k: round(v, 4) for k, v in result.items()}

    return result

Checking model translation without model training

In [ ]:
from transformers import pipeline

translator = pipeline("translation_en_to_es", model = checkpoint)

translator

### TODO Recording:

- Replace the text each time to show the translation, all translations will be poor

In [ ]:
#text = "translate English to Spanish: It was very bad. "

# text = "translate English to Spanish: Why are you silent? "

# text = "translate English to Spanish: Don't you think the weather is great today? "
text= "translate English to Spanish: I live in Rome "

translator(text, max_new_tokens=40)

[{'translation_text': 'Ich lebe in Rom'}]

To finetune a model in TensorFlow, we start by setting up an optimizer function, learning rate schedule, and some training hyperparameters:

In [ ]:
from transformers import AdamWeightDecay

optimizer = AdamWeightDecay(learning_rate = 2e-5, weight_decay_rate = 0.01)

We can load T5 with TFAutoModelForSeq2SeqLM

In [ ]:
from transformers import TFAutoModelForSeq2SeqLM

model = TFAutoModelForSeq2SeqLM.from_pretrained(checkpoint)

All PyTorch model weights were used when initializing TFT5ForConditionalGeneration.

All the weights of TFT5ForConditionalGeneration were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFT5ForConditionalGeneration for predictions without further training.


Converting  datasets to the tf.data.Dataset format with prepare_tf_dataset():

In [ ]:
tf_train_set = model.prepare_tf_dataset(
    tokenized_eng_spa_ds["train"],
    shuffle = True,
    batch_size = 64,
    collate_fn = data_collator,
)

tf_validation_set = model.prepare_tf_dataset(
    tokenized_eng_spa_ds["test"],
    shuffle = False,
    batch_size = 64,
    collate_fn = data_collator,
)

Configuring the model for training with compile. Note that Transformers models all have a default task-relevant loss function, so we don’t need to specify one unless we want to:

In [ ]:
import tensorflow as tf

model.compile(optimizer = optimizer)

model.summary()

Model: "tft5_for_conditional_generation_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 shared (Embedding)          multiple                  16449536  
                                                                 
 encoder (TFT5MainLayer)     multiple                  35330816  
                                                                 
 decoder (TFT5MainLayer)     multiple                  41625344  
                                                                 
Total params: 60506624 (230.81 MB)
Trainable params: 60506624 (230.81 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


predict_with_generate - Whether we should use model.generate() to get outputs for the model.

In [ ]:
from transformers.keras_callbacks import KerasMetricCallback

metric_callback = KerasMetricCallback(metric_fn = compute_metrics, eval_dataset = tf_validation_set, predict_with_generate = True)

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

Specifying where to push your model and tokenizer in the PushToHubCallback

In [ ]:
from transformers.keras_callbacks import PushToHubCallback

push_to_hub_callback = PushToHubCallback(
    output_dir = "english-to-spanish-lang-translation-model",
    tokenizer = tokenizer,
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_deprecation.py:131: FutureWarning: 'Repository' (from 'huggingface_hub.repository') is deprecated and will be removed from version '1.0'. Please prefer the http-based alternatives instead. Given its large adoption in legacy code, the complete removal is only planned on next major release.
For more details, please read https://huggingface.co/docs/huggingface_hub/concepts/git_vs_http.
  warnings.warn(warning_message, FutureWarning)
/content/english-to-spanish-lang-translation-model is already a clone of https://huggingface.co/jinxxx123/english-to-spanish-lang-translation-model. Make sure you pull the latest changes with `repo.git_pull()`.


Then bundling callbacks together:

In [ ]:
callbacks = [metric_callback, push_to_hub_callback]

Finally, we are ready to start training your model! Calling fit with your training and validation datasets, the number of epochs, and your callbacks to finetune the model

In [ ]:
model.fit(x = tf_train_set, validation_data = tf_validation_set, epochs = 3, callbacks = callbacks)

Epoch 1/3
892/892 [==============================] - ETA: 0s - loss: 1.7670

/usr/local/lib/python3.10/dist-packages/transformers/generation/tf_utils.py:838: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length.  recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


892/892 [==============================] - 720s 789ms/step - loss: 1.7670 - val_loss: 1.1178 - bleu: 2.0633 - gen_len: 16.4917
Epoch 2/3
892/892 [==============================] - ETA: 0s - loss: 1.1947

/usr/local/lib/python3.10/dist-packages/transformers/generation/tf_utils.py:838: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length.  recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


892/892 [==============================] - 700s 785ms/step - loss: 1.1947 - val_loss: 0.9787 - bleu: 3.7341 - gen_len: 16.2275
Epoch 3/3
892/892 [==============================] - ETA: 0s - loss: 1.0799

/usr/local/lib/python3.10/dist-packages/transformers/generation/tf_utils.py:838: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length.  recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


892/892 [==============================] - 699s 785ms/step - loss: 1.0799 - val_loss: 0.8758 - bleu: 5.9705 - gen_len: 16.0262


In [ ]:
translator = pipeline("translation", model = "english-to-spanish-lang-translation-model", max_new_tokens=20)

translator

All model checkpoint layers were used when initializing TFT5ForConditionalGeneration.

All the layers of TFT5ForConditionalGeneration were initialized from the model checkpoint at english-to-spanish-lang-translation-model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFT5ForConditionalGeneration for predictions without further training.
/usr/local/lib/python3.10/dist-packages/transformers/pipelines/__init__.py:1049: UserWarning: "translation" task was used, instead of "translation_XX_to_YY", defaulting to "translation_en_to_de"
  warnings.warn(


### TODO Recording:

- Please record one text at a time and show the translation (only the next two cells)

In [ ]:
text = "translate English to Spanish: The movie was excellent "
#text = "translate English to Spanish: It was very bad. "

# text = "translate English to Spanish: Why are you silent? "


# text = "translate English to Spanish: Don't you think the weather is great today? "
# text= "translate English to Spanish: I live in Rome "
#text= "translate English to Spanish: Where is your friend? "

In [ ]:
translator(text)

[{'translation_text': 'El filmo es excelente'}]

Alternatively, We can generate translation this way

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("english-to-spanish-lang-translation-model")
inputs = tokenizer(text, return_tensors = "tf").input_ids

In [ ]:
from transformers import TFAutoModelForSeq2SeqLM

model = TFAutoModelForSeq2SeqLM.from_pretrained("english-to-spanish-lang-translation-model")
outputs = model.generate(inputs, max_new_tokens = 40, do_sample = True, top_k = 30, top_p = 0.95)

All model checkpoint layers were used when initializing TFT5ForConditionalGeneration.

All the layers of TFT5ForConditionalGeneration were initialized from the model checkpoint at english-to-spanish-lang-translation-model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFT5ForConditionalGeneration for predictions without further training.


In [ ]:
tokenizer.decode(outputs[0], skip_special_tokens = True)

'El filmo era exzellento'